# タイタニック生存者予測モデル

[Titanic: Machine Learning from Disaster](https://www.kaggle.com/c/titanic)

以下の内容は、九大OB のT君が学部生時代に行った分析過程の一部です。  
どうアプローチしたら良いのかわからない人は参考にしてください。

---

---

まずはどんな問題なのか把握しないとな。  
なるほど、生死がわかっている人たちのデータ (train.csv) からモデルを立てて、生死が隠されているの人たち (test.csv) の生死を予測すれば良いわけだな。

まぁ、とりあえずはデータを読み込んでみよう。

In [1]:
using DataFrames, StatsBase, StatPlots
gr()

train = readtable("train.csv")
head(train)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NA,S
2,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
3,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.925,NA,S
4,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1,C123,S
5,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.05,NA,S
6,6,0,3,"Moran, Mr. James",male,NA,0,0,330877,8.4583,NA,Q


In [2]:
test = readtable("test.csv")
head(test)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NA,Q
2,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0,NA,S
3,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NA,Q
4,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NA,S
5,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NA,S
6,897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.225,NA,S


サイトによると各列のデータの意味は次のとおりと。

| Variable | Definition                                 | Key                                            |
|----------|--------------------------------------------|------------------------------------------------|
| survival | Survival                                   | 0 = No, 1 = Yes                                |
| pclass   | Ticket class                               | 1 = 1st, 2 = 2nd, 3 = 3rd                      |
| sex      | Sex                                        |                                                |
| Age      | Age in years                               |                                                |
| sibsp    | # of siblings / spouses aboard the Titanic |                                                |
| parch    | # of parents / children aboard the Titanic |                                                |
| ticket   | Ticket number                              |                                                |
| fare     | Passenger fare                             |                                                |
| cabin    | Cabin number                               |                                                |
| embarked | Port of Embarkation                        | C = Cherbourg, Q = Queenstown, S = Southampton |

パラメータが多くてこれは大変そうだ。どうアプローチしたもんかなぁ。<br>
名前で生死に差がでるとは思えないから Name 列でも消しとくかな。

In [3]:
delete!(train, :Name)
head(train)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,1,0,3,male,22.0,1,0,A/5 21171,7.25,NA,S
2,2,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C
3,3,1,3,female,26.0,0,0,STON/O2. 3101282,7.925,NA,S
4,4,1,1,female,35.0,1,0,113803,53.1,C123,S
5,5,0,3,male,35.0,0,0,373450,8.05,NA,S
6,6,0,3,male,NA,0,0,330877,8.4583,NA,Q


生存者と死者にはそれぞれ共通する傾向があるかもしれないからデータを分けておこう！

In [4]:
survived = train[train[:Survived] .== 1, :]
delete!(survived, :Survived) # 生死はわかっているので Survived 列を削除
head(survived)

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,2,1,female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,female,26.0,0,0,STON/O2. 3101282,7.925,NA,S
3,4,1,female,35.0,1,0,113803,53.1,C123,S
4,9,3,female,27.0,0,2,347742,11.1333,NA,S
5,10,2,female,14.0,1,0,237736,30.0708,NA,C
6,11,3,female,4.0,1,1,PP 9549,16.7,G6,S


In [5]:
dead = train[train[:Survived] .== 0, :]
delete!(dead, :Survived) # 生死はわかっているので Survived 列を削除
head(dead)

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,1,3,male,22.0,1,0,A/5 21171,7.25,NA,S
2,5,3,male,35.0,0,0,373450,8.05,NA,S
3,6,3,male,NA,0,0,330877,8.4583,NA,Q
4,7,1,male,54.0,0,0,17463,51.8625,E46,S
5,8,3,male,2.0,3,1,349909,21.075,NA,S
6,13,3,male,20.0,0,0,A/5. 2151,8.05,NA,S


タイタニックといえばイギリスの客船だったなぁ。イギリスといえば英国紳士。レディーファーストの文化が根付いてるから、女性の方が優先的に救命ボートに乗ってそうだな。よし、生存者と死者の男女の数を調べてみよう。

In [6]:
# データ全体の男女の数
countmap(train[:Sex])

Dict{Union{DataArrays.NAtype,String},Int64} with 2 entries:
  "male"   => 577
  "female" => 314

In [7]:
# 生存者の男女の数
countmap(survived[:Sex])

Dict{Union{DataArrays.NAtype,String},Int64} with 2 entries:
  "male"   => 109
  "female" => 233

In [8]:
# 死者の男女の数
countmap(dead[:Sex])

Dict{Union{DataArrays.NAtype,String},Int64} with 2 entries:
  "male"   => 468
  "female" => 81

In [9]:
# 男女別の生存者の割合
collect(values(countmap(survived[:Sex]))) ./ collect(values(countmap(train[:Sex])))

2-element Array{Float64,1}:
 0.188908
 0.742038

In [10]:
groupedbar([countmap(train[:Sex])["male"] countmap(train[:Sex])["female"]; countmap(dead[:Sex])["male"] countmap(dead[:Sex])["female"]],
            label = ["male" "female"],
            bar_position = :stack, 
            bar_width=0.7,
            xticks = (1:2, ["survived", "dead"]),
            leg = true)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 survived 
 
 
 dead 
 
 
 0 
 
 
 250 
 
 
 500 
 
 
 750 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 male 
 
 
 
 
 female

睨んだ通り、男性よりも女性の方の生存確率が高そうだ。

まだ Kaggle に慣れてないし、練習がてら最初は
> 男性は皆亡くなって、女性は皆生き残る

という大雑把なモデルの結果で提出してみよう。

In [11]:
ID = zeros(Int, size(test, 1))
live = copy(ID);
for (idx, row) in enumerate(eachrow(test))
    ID[idx] = row[:PassengerId]
    if row[:Sex] == "female"
        live[idx] = 1
    end
end

In [12]:
submit = DataFrame(PassengerId = ID, Survived = live)
head(submit)

,PassengerId,Survived
1,892,0
2,893,1
3,894,0
4,895,0
5,896,1
6,897,0


In [13]:
writetable("submit.csv", submit)

さてさて、こんな安直なモデルでどこまでの精度が出るのかな？

![KaggleTitanic](kaggle_titanic.png "titanic")

おぉ!? こんなモデルでも 76 % くらいの精度か。順位は 5834人中の4267位。精度が 80 % 超えると上位 10 % に入れるようだから、ここから 1 % 上げるのは相当難しいってことか。

次は何を基準にするかな。若い人のほうが体力あるから生き残れそうだなよなぁ。船は真っ二つに分裂したから客室によっても変わりそうだ ・・・

---

---

アプローチの仕方はわかったでしょうか?  
それでは皆さんも試行錯誤を重ね、自分なりのモデルを構築してみてください。